In [8]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_stock_prices.csv to cleaned_stock_prices (2).csv


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
import zipfile
from google.colab import files
df = pd.read_csv("cleaned_stock_prices.csv")
target = "Price"
def convert_volume(value):
    if isinstance(value, str):
        value = value.replace(",", "").strip()
        if value.endswith("M"):
            return float(value[:-1]) * 1_000_000
        elif value.endswith("K"):
            return float(value[:-1]) * 1_000
        elif value == "-" or value == "":
            return 0
        else:
            return float(value)
    return value
if "Vol." in df.columns:
    df["Vol."] = df["Vol."].apply(convert_volume)

def clean_percentage(x):
    if isinstance(x, str):
        x = x.replace("%", "").strip()
        return float(x) / 100
    return x
if "Change %" in df.columns:
    df["Change %"] = df["Change %"].apply(clean_percentage)

if "Date" in df.columns:
    df = df.drop(columns=["Date"])
X = df.drop(columns=[target])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
ann = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(1)
])
ann.compile(optimizer='adam', loss='mse')
ann.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
results = pd.DataFrame({
    "Model": ["Linear Regression", "Random Forest", "ANN"],
    "MSE": [
        mean_squared_error(y_test, y_pred_lr),
        mean_squared_error(y_test, y_pred_rf),
        mean_squared_error(y_test, ann.predict(X_test))
    ],
    "R2 Score": [
        r2_score(y_test, y_pred_lr),
        r2_score(y_test, y_pred_rf),
        r2_score(y_test, ann.predict(X_test))
    ]
})
print("\n=== MODEL PERFORMANCE COMPARISON ===")
print(results)
with zipfile.ZipFile("regression_outputs.zip", "w") as zipf:
    results.to_csv("regression_model_results.csv", index=False)
    zipf.write("regression_model_results.csv")
    joblib.dump(lr, "linear_regression_model.pkl")
    zipf.write("linear_regression_model.pkl")
    joblib.dump(rf, "random_forest_model.pkl")
    zipf.write("random_forest_model.pkl")
    ann.save("ann_model.h5")
    zipf.write("ann_model.h5")
files.download("regression_outputs.zip")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - loss: 173987.7812 - val_loss: 172777.9062
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 174362.7656 - val_loss: 172626.9219
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 173355.7031 - val_loss: 172472.8906
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 173847.9219 - val_loss: 172308.7969
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 172811.2656 - val_loss: 172131.2188
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 173160.9844 - val_loss: 171937.0938
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 172928.0156 - val_loss: 171722.3281
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 174540.2344 - val_loss: 171485.2969
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 174088.3281 - val_loss: 171215.2188
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 172623.1406 - val_loss: 170907.8281
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 172918.8906

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>